## RL Env

In [1]:
import numpy as np
from flow.envs.base import Env

In [2]:
class DensityAwareBottleneckEnv(Env):
    def __init__(self, env_params, sim_params, network, simulator='traci'):
        for p in ADDITIONAL_ENV_PARAMS.keys():
            if p not in env_params.additional_params:
                raise KeyError(
                    'Environment parameter "{}" not supplied'.format(p))

        super().__init__(env_params, sim_params, network, simulator)
        
    @property
    def action_space(self):
        """See class definition."""
        return Box(
            low=-float("inf"),
            high=float("inf"),
            shape=(1, ),
            dtype=np.float32)

    @property
    def observation_space(self):
        """See class definition."""
        return Box(
            low=-float("inf"),
            high=float("inf"),
            shape=(1, ),
            dtype=np.float32)
    
    def _apply_rl_actions(self, rl_actions):
        """See parent class.

        To be implemented by child classes.
        """
        pass

    def compute_reward(self, rl_actions, **kwargs):
        """Outflow rate over last ten seconds normalized to max of 1."""
        veh_ids = self.k.vehicle.get_ids()
        speeds = self.k.vehicle.get_speed(veh_ids)
        avg_speed = np.mean(speeds)
        return avg_speed

    def get_state(self):
        """See class definition."""
        pass

In [3]:
from flow.core.params import SumoParams, EnvParams, InitialConfig, VehicleParams, InFlows, NetParams, TrafficLightParams
from flow.controllers import ContinuousRouter, SimLaneChangeController
from flow.core.params import SumoCarFollowingParams, SumoLaneChangeParams

# Ours
from custom_networks import CustomBottleneckNetwork

### Network Config 

In [4]:
HORIZON = 5000

N_CPUS = 1
N_ROLLOUTS = N_CPUS * 4 # Per iteration

In [5]:
vehicles = VehicleParams()

# Randomly add some initializing human vehicles
# Add a few vehicles to initialize the simulation. The vehicles have all lane changing enabled, 
# which is mode 1621
vehicles.add(
    veh_id="human",
    lane_change_controller=(SimLaneChangeController, {}),
    routing_controller=(ContinuousRouter, {}),
    car_following_params=SumoCarFollowingParams(
        speed_mode=7, # all_checks, 25, 31
    ),
    lane_change_params=SumoLaneChangeParams(
        lane_change_mode=1621,
    ),
    num_vehicles=1)
    

In [6]:
ADDITIONAL_NET_PARAMS = {
    "speed_limit": 20,
}

additional_env_params = {
    "target_velocity": 40,
    "max_accel": 1,
    "max_decel": 1,
    "lane_change_duration": 5,
    "add_rl_if_exit": False,
    "disable_tb": True,
    "disable_ramp_metering": True,
}
ADDITIONAL_ENV_PARAMS = {}
additional_net_params = ADDITIONAL_NET_PARAMS.copy()

In [7]:
inflow = InFlows()
inflow.add(
    veh_type="human",
    edge="edge0",
    vehs_per_hour= 10000,
    depart_lane="random",
    depart_speed=10)

inflow.add(
    veh_type="human",
    edge="edge1",
    vehs_per_hour= 10000,
    depart_lane="random",
    depart_speed=10)

traffic_lights = TrafficLightParams()

In [8]:
flow_params = dict(
    exp_tag = "DensityAwareBottleneck",
    env_name=DensityAwareBottleneckEnv,
    network=CustomBottleneckNetwork,
    simulator='traci',
    sim = SumoParams(sim_step=0.1, 
                        render=True,
                        restart_instance=True),
    env=EnvParams(
        warmup_steps=8000,
        sims_per_step=1,
        horizon=HORIZON,
        additional_params=additional_env_params,
    ),
    net = NetParams(
    inflows=inflow,
    additional_params=additional_net_params),
    veh=vehicles,
    initial=InitialConfig(
        spacing="uniform",
        min_gap=5,
        lanes_distribution=float("inf"),
        #edges_distribution=["2", "3", "4", "5"],
    ),
    tls=traffic_lights
)

In [9]:
from flow.core.experiment import Experiment

exp = Experiment(flow_params)

# run 
_ = exp.run(1)

FatalTraCIError: connection closed by SUMO

In [ ]:
# run_exp(flow_rate=10000, scaling=2, disable_tb=True, disable_ramp_meter=True)

In [ ]:
# from flow.core.params import SumoParams, EnvParams, NetParams, InitialConfig, \
#     InFlows, SumoLaneChangeParams, SumoCarFollowingParams
# from flow.core.params import VehicleParams
# from flow.core.params import TrafficLightParams

# from flow.networks.bottleneck import BottleneckNetwork
# from flow.controllers import SimLaneChangeController, ContinuousRouter
# from flow.envs.bottleneck import BottleneckEnv
# from flow.core.experiment import Experiment

# import logging

# def run_exp(flow_rate,
#             scaling=1,
#             disable_tb=True,
#             disable_ramp_meter=True,
#             n_crit=1000,
#             feedback_coef=20):
#     # Set up SUMO to render the results, take a time_step of 0.5 seconds per simulation step
#     sim_params = SumoParams(
#         sim_step=0.5,
#         render=True,
#         overtake_right=False,
#         restart_instance=False)

#     vehicles = VehicleParams()

#     # Add a few vehicles to initialize the simulation. The vehicles have all lane changing enabled, 
#     # which is mode 1621
#     vehicles.add(
#         veh_id="human",
#         lane_change_controller=(SimLaneChangeController, {}),
#         routing_controller=(ContinuousRouter, {}),
#         car_following_params=SumoCarFollowingParams(
#             speed_mode=25,
#         ),
#         lane_change_params=SumoLaneChangeParams(
#             lane_change_mode=1621,
#         ),
#         num_vehicles=1)

#     # These are additional params that configure the bottleneck experiment. They are explained in more
#     # detail below.
#     additional_env_params = {
#         "target_velocity": 40,
#         "max_accel": 1,
#         "max_decel": 1,
#         "lane_change_duration": 5,
#         "add_rl_if_exit": False,
#         "disable_tb": disable_tb,
#         "disable_ramp_metering": disable_ramp_meter,
#         "n_crit": n_crit,
#         "feedback_coeff": feedback_coef,
#     }
#     # Set up the experiment to run for 1000 time steps i.e. 500 seconds (1000 * 0.5)
#     env_params = EnvParams(
#         horizon=10000, additional_params=additional_env_params)

#     # Add vehicle inflows at the front of the bottleneck. They enter with a flow_rate number of vehicles 
#     # per hours and with a speed of 10 m/s
#     inflow = InFlows()
#     inflow.add(
#         veh_type="human",
#         edge="1",
#         vehsPerHour=flow_rate,
#         departLane="random",
#         departSpeed=10)

#     # Initialize the traffic lights. The meanings of disable_tb and disable_ramp_meter are discussed later.
#     traffic_lights = TrafficLightParams()
#     if not disable_tb:
#         traffic_lights.add(node_id="2")
#     if not disable_ramp_meter:
#         traffic_lights.add(node_id="3")

#     additional_net_params = {"scaling": scaling, "speed_limit": 23}
#     net_params = NetParams(
#         inflows=inflow,
#         additional_params=additional_net_params)

#     initial_config = InitialConfig(
#         spacing="random",
#         min_gap=5,
#         lanes_distribution=float("inf"),
#         edges_distribution=["2", "3", "4", "5"])

#     flow_params = dict(
#         exp_tag='bay_bridge_toll',
#         env_name=BottleneckEnv,
#         network=BottleneckNetwork,
#         simulator='traci',
#         sim=sim_params,
#         env=env_params,
#         net=net_params,
#         veh=vehicles,
#         initial=initial_config,
#         tls=traffic_lights,
#     )

#     # number of time steps
#     flow_params['env'].horizon = 1000
#     exp = Experiment(flow_params)

#     # run the sumo simulation
#     _ = exp.run(1)